# Climatiq Technical Exercise 

#### Aim of the Exercise
The aim of this exercise is to extract the first two entries of Table 9 from the EPA excel file and create a table which will be structured like the Open Emission Factors Database OEFDB. These first two entries include 6 values which constitute emission factors for metal material (Aluminium Cans and Aluminium Ingot) and their respected carbon footprint when Recycled, Landfilled and Composted. 

#### Methodology
We start by creating an empty dataframe structured like the OEFDB. To do so we read the headers of the columns from  OpenEmissionFactorsDB.csv file provided from https://github.com/climatiq/Open-Emission-Factors-DB/. 
The EPA excel file is quite complicated so we just copy the Table 9, which is the table we are interested in, in a new excel file. We read this file, create a dataframe and select the values that we would like to work on. 
We calculate the carbon dioxide equivalent ${CO_2}e$ my multiplying the emission factors by 1000.
Then, we follow the guidelines as suggested in documentation  https://github.com/climatiq/Open-Emission-Factors-DB/blob/main/Sector-Category-ID_structure.csv (DATA_GUIDANCE.md, Sector-categoryid_structure.csv files) and of course check table 9 to start filling out the rows and columns with the appropriate values in an automated way (please check the table below "Variables for each item in our Task"). To do so, the most efficient way was to create a dictionary and in the end append the values in the dataframe structured like the OEFDB.
I decided not to create a pull request since the data already exist in the database, but the notebook and the excel files used will be found on my github account. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
headers_df = pd.read_csv('OpenEmissionFactorsDB.csv', nrows=1).columns
headers_df

Index(['UUID', 'sector', 'category', 'activity_id', 'name', 'activity_unit',
       'kgCO2e-AR5', 'kgCO2e-AR4', 'kgCO2', 'kgCH4', 'kgN2O',
       'kgCO2e-OtherGHGs-AR5', 'kgCO2e-OtherGHGs-AR4', 'uncertainty', 'scope',
       'lca_activity', 'source', 'year_released', 'years_valid',
       'years_calculated_from', 'region', 'data_quality', 'contributor',
       'date_accessed', 'description', 'source_link'],
      dtype='object')

In [3]:
df = pd.DataFrame(columns = headers_df)
df

,UUID,sector,category,activity_id,name,activity_unit,kgCO2e-AR5,kgCO2e-AR4,kgCO2,kgCH4,...,source,year_released,years_valid,years_calculated_from,region,data_quality,contributor,date_accessed,description,source_link


In [4]:
table_9_df = pd.read_excel('Climatiq_task_Table_9.xlsx')

In [5]:
table_9_df.head()

,Material,Recycled,Landfilled,Combusted,Composted,Anaerobically Digested (Dry Digestate with Curing),Anaerobically Digested (Wet Digestate with Curing)
0,Aluminum Cans,0.06,0.02,0.01,NaN,NaN,NaN
1,Aluminum Ingot,0.04,0.02,0.01,NaN,NaN,NaN
2,Steel Cans,0.32,0.02,0.01,NaN,NaN,NaN
3,Copper Wire,0.18,0.02,0.01,NaN,NaN,NaN
4,Glass,0.05,0.02,0.01,NaN,NaN,NaN


In [6]:
table_9_df = table_9_df.iloc[:2, 0:4]

In [7]:
table_9_df

,Material,Recycled,Landfilled,Combusted
0,Aluminum Cans,0.06,0.02,0.01
1,Aluminum Ingot,0.04,0.02,0.01


$$ \text { emission factor } = { \text { Metric Tons } {CO_{2}e}  \over \text { Short Ton Material }} $$

In [8]:
table_9_df[table_9_df.select_dtypes(include=['number']).columns] *= 1000 

In [9]:
table_9_df

,Material,Recycled,Landfilled,Combusted
0,Aluminum Cans,60.0,20.0,10.0
1,Aluminum Ingot,40.0,20.0,10.0


Create acticity_id indicator and names of the emission factors as suggested in documentation: https://github.com/climatiq/Open-Emission-Factors-DB/blob/main/Sector-Category-ID_structure.csv, also interpret kgCO2e-AR4 values.

In [10]:
activity_names = []
names = []
values = []
for i in range(len(table_9_df)):
    name = table_9_df['Material'][i]
    for j in range(1, len(table_9_df.columns)):
        disposal_method = table_9_df.columns[j]
        disposal_method = disposal_method.lower()
        name_string = f'{name} - {disposal_method}'
        activity_id_string = f'waste_type_{name.replace(" ", "_").lower()}-disposal_method_{disposal_method}'
        names.append(name_string)
        activity_names.append(activity_id_string)
        values.append(table_9_df[table_9_df.columns[j]][i])

In [11]:
data_rows = len(table_9_df)
data_columns = len(table_9_df.columns) - 1
nrows = data_rows*data_columns
headers_df = pd.read_csv('OpenEmissionFactorsDB.csv', nrows=1).columns
df = pd.DataFrame(columns = headers_df)

In [12]:
df

,UUID,sector,category,activity_id,name,activity_unit,kgCO2e-AR5,kgCO2e-AR4,kgCO2,kgCH4,...,source,year_released,years_valid,years_calculated_from,region,data_quality,contributor,date_accessed,description,source_link


### Variables for each item in our Task

| Variable |  Value  |         Source         | Same for all? |
|:--------:|:-------:|:----------------------:|:-------------:|
|  sector  | Waste | Table 9  |      YES      |
| category | Metal Waste | Sector-Category-ID_structure.csv | YES |
|  activity_id | _varies_        |           Table 9              |NO|
|    name     |    _varies_     |     Sector-Category-ID_structure.csv                   |       NO        |
|    activity_unit     |     short ton    |        Table 9                |   YES            |
| kgCO2e-AR4 | _varies_ | Table 9 | NO |
| kgCO2e-AR5 | not supplied | not available in Table 9 | YES |
| kgCO2 | not supplied | not available in Table 9 | YES |
| kgCH4 | not supplied | not available in Table 9 | YES |
| kgN2O | not supplied | not available in Table 9 | YES |
| kgCO2e-OtherGHGs-AR5 | not supplied | not available in Table 9 | YES |
| kgCO2e-OtherGHGs-AR4 | not supplied | not available in Table 9 | YES |
| uncertainty | not supplied  | not available in Table 9 | YES |
| scope | 3 | Table 9 - Notes | YES |
| lca_activity | end_of_life | Table 9 & Data Guidance | YES|
| source| EPA | Table 9 | YES|
| year_released | 2021 | EPA excel file | YES|
| years_valid| 2021 | EPA excel file | YES|
| years_calculated_from | not supplied | not available in EPA excel file | YES |
| region | USA | EPA excel file | YES |
| data_quality| | | YES|
| contrinutor | Georgia Pantelidou | User | YES|
| date_accessed | 2022/09/08 | Data Guidance - format YY/MM/DD | YES |
| description | _varies_ | Table 9 - Notes  | YES |
| source_link | https://www.epa.gov/climateleadership/ghg-emission-factors-hub | EPA excel file | YES |




In [13]:
description_list = ['Emission intensity of recycling the specified material. These factors do not include any avoided emissions impact from any of the disposal methods. Recycling emissions include transport to recycling facility and sorting of recycled materials at material recovery facility. Retrieved from the EPAs GHG Emission Factors Hub (xlsx).', 
                    'Emission intensity of disposing of the specified material to landfill. These factors do not include any avoided emissions impact from any of the disposal methods. Landfilling emissions include transport to landfill/equipment use at landfill and fugitive landfill CH4 emissions. Landfill CH4 is based on typical landfill gas collection practices and average landfill moisture conditions. Retrieved from the EPAs GHG Emission Factors Hub (xlsx).', 
                    'Emission intensity of disposing of the specified material through cumbustion.  These factors do not include any avoided emissions impact from any of the disposal methods.  Combustion emissions include transport to combustion facility and combustion-related non-biogenic CO2 and N2O.  Retrieved from the EPAs GHG Emission Factors Hub (xlsx).', 
                    'Emission intensity of recycling the specified material. These factors do not include any avoided emissions impact from any of the disposal methods. Recycling emissions include transport to recycling facility and sorting of recycled materials at material recovery facility. Retrieved from the EPAs GHG Emission Factors Hub (xlsx).', 
                    'Emission intensity of disposing of the specified material to landfill. These factors do not include any avoided emissions impact from any of the disposal methods. Landfilling emissions include transport to landfill/equipment use at landfill and fugitive landfill CH4 emissions. Landfill CH4 is based on typical landfill gas collection practices and average landfill moisture conditions. Retrieved from the EPAs GHG Emission Factors Hub (xlsx).', 
                    'Emission intensity of disposing of the specified material through cumbustion. These factors do not include any avoided emissions impact from any of the disposal methods. Combustion emissions include transport to combustion facility and combustion-related non-biogenic CO2 and N2O. Retrieved from the EPAs GHG Emission Factors Hub (xlsx).']

UUID_list = ['4ac0f5f1-4044-4e54-916d-fa93c91c3005', '2ff6d913-c472-4d47-a950-639c9a82d3a7', '96a37944-dfd8-4e13-9452-ac7c5a7da0c5', 
'3241beb9-38f5-47b1-bbb6-d162c9705a96', '7328d4a1-fe01-46cc-9864-28903b41b514', '750ede40-5538-4d3b-a018-db91843b7a18']

Convert the empty dataframe to an empty dictionary and start filling it, at the end we append the values to the dataframe.

In [14]:
row_dictionary = df.to_dict()

In [15]:
for i in range(nrows):
    row_dictionary['UUID'] = UUID_list[i]
    row_dictionary['sector'] = 'Waste'
    row_dictionary['category'] = 'Metal Waste'
    row_dictionary['activity_id'] = activity_names[i]
    row_dictionary['name'] = names[i]
    row_dictionary['activity_unit'] = 'short ton'
    row_dictionary['kgCO2e-AR4'] = values[i]
    row_dictionary['kgCO2e-AR5'] = 'not-supplied'
    row_dictionary['kgCO2'] = 'not supplied'
    row_dictionary['kgCH4'] = 'not supplied'
    row_dictionary['kgN2O'] = 'not supplied'
    row_dictionary['kgCO2e-OtherGHGs-AR5'] = 'not supplied'
    row_dictionary['kgCO2e-OtherGHGs-AR4'] = 'not supplied'
    row_dictionary['uncertainty'] = 'not supplied'
    row_dictionary['scope'] = '3'
    row_dictionary['lca_activity'] = 'end_of_life'
    row_dictionary['source'] = 'EPA'
    row_dictionary['year_released'] = 2021
    row_dictionary['years_valid'] = 2021
    row_dictionary['years_calculated_from'] = 'not supplied'
    row_dictionary['region'] = 'USA'
    row_dictionary['data_quality'] = ' '
    row_dictionary['contributor'] = 'Georgia Pantelidou'
    row_dictionary['date_accessed'] = '2022/09/08'
    row_dictionary['description'] = description_list[i]
    row_dictionary['source_link'] = 'https://www.epa.gov/climateleadership/ghg-emission-factors-hub'
    df = df.append(row_dictionary,ignore_index=True)

In [16]:
df

,UUID,sector,category,activity_id,name,activity_unit,kgCO2e-AR5,kgCO2e-AR4,kgCO2,kgCH4,...,source,year_released,years_valid,years_calculated_from,region,data_quality,contributor,date_accessed,description,source_link
0,4ac0f5f1-4044-4e54-916d-fa93c91c3005,Waste,Metal Waste,waste_type_aluminum_cans-disposal_method_recycled,Aluminum Cans - recycled,short ton,not-supplied,60.0,not supplied,not supplied,...,EPA,2021,2021,not supplied,USA,,Georgia Pantelidou,2022/09/08,Emission intensity of recycling the specified ...,https://www.epa.gov/climateleadership/ghg-emis...
1,2ff6d913-c472-4d47-a950-639c9a82d3a7,Waste,Metal Waste,waste_type_aluminum_cans-disposal_method_landf...,Aluminum Cans - landfilled,short ton,not-supplied,20.0,not supplied,not supplied,...,EPA,2021,2021,not supplied,USA,,Georgia Pantelidou,2022/09/08,Emission intensity of disposing of the specifi...,https://www.epa.gov/climateleadership/ghg-emis...
2,96a37944-dfd8-4e13-9452-ac7c5a7da0c5,Waste,Metal Waste,waste_type_aluminum_cans-disposal_method_combu...,Aluminum Cans - combusted,short ton,not-supplied,10.0,not supplied,not supplied,...,EPA,2021,2021,not supplied,USA,,Georgia Pantelidou,2022/09/08,Emission intensity of disposing of the specifi...,https://www.epa.gov/climateleadership/ghg-emis...
3,3241beb9-38f5-47b1-bbb6-d162c9705a96,Waste,Metal Waste,waste_type_aluminum_ingot-disposal_method_recy...,Aluminum Ingot - recycled,short ton,not-supplied,40.0,not supplied,not supplied,...,EPA,2021,2021,not supplied,USA,,Georgia Pantelidou,2022/09/08,Emission intensity of recycling the specified ...,https://www.epa.gov/climateleadership/ghg-emis...
4,7328d4a1-fe01-46cc-9864-28903b41b514,Waste,Metal Waste,waste_type_aluminum_ingot-disposal_method_land...,Aluminum Ingot - landfilled,short ton,not-supplied,20.0,not supplied,not supplied,...,EPA,2021,2021,not supplied,USA,,Georgia Pantelidou,2022/09/08,Emission intensity of disposing of the specifi...,https://www.epa.gov/climateleadership/ghg-emis...
5,750ede40-5538-4d3b-a018-db91843b7a18,Waste,Metal Waste,waste_type_aluminum_ingot-disposal_method_comb...,Aluminum Ingot - combusted,short ton,not-supplied,10.0,not supplied,not supplied,...,EPA,2021,2021,not supplied,USA,,Georgia Pantelidou,2022/09/08,Emission intensity of disposing of the specifi...,https://www.epa.gov/climateleadership/ghg-emis...


#### For the rest of the data of Table 9, all we need to do is change line 33 to add more rows to the database, with the exception of sector, category and description columns which have to be interpreted manually.

In [17]:
row_dictionary

{'UUID': '750ede40-5538-4d3b-a018-db91843b7a18',
 'sector': 'Waste',
 'category': 'Metal Waste',
 'activity_id': 'waste_type_aluminum_ingot-disposal_method_combusted',
 'name': 'Aluminum Ingot - combusted',
 'activity_unit': 'short ton',
 'kgCO2e-AR5': 'not-supplied',
 'kgCO2e-AR4': 10.0,
 'kgCO2': 'not supplied',
 'kgCH4': 'not supplied',
 'kgN2O': 'not supplied',
 'kgCO2e-OtherGHGs-AR5': 'not supplied',
 'kgCO2e-OtherGHGs-AR4': 'not supplied',
 'uncertainty': 'not supplied',
 'scope': '3',
 'lca_activity': 'end_of_life',
 'source': 'EPA',
 'year_released': 2021,
 'years_valid': 2021,
 'years_calculated_from': 'not supplied',
 'region': 'USA',
 'data_quality': ' ',
 'contributor': 'Georgia Pantelidou',
 'date_accessed': '2022/09/08',
 'description': 'Emission intensity of disposing of the specified material through cumbustion. These factors do not include any avoided emissions impact from any of the disposal methods. Combustion emissions include transport to combustion facility and co